In [1]:
!pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

In [2]:
!pip install --upgrade --quiet sqlalchemy chromadb

In [3]:
for line in open('openai_key', 'r'):
    key = line.strip()

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


In [18]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [10]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='./teams2.csv')
data = loader.load()

In [13]:

prompt = ChatPromptTemplate.from_template("Which project is 'Sentient Sentences' working on?")
model = ChatOpenAI(openai_api_key = key)
chain = prompt | model

result = chain.invoke({})
result.content

"'Sentient Sentences' is working on developing a language model trained on a diverse range of text data to generate more intelligent and contextually relevant sentences."

In [14]:

prompt = ChatPromptTemplate.from_template("Which project is Tingyu Shi working on?")
model = ChatOpenAI(openai_api_key = key)
chain = prompt | model

result = chain.invoke({})
result.content

'Tingyu Shi is currently working on the project "Development of Machine Learning Algorithms for Predictive Modeling in Healthcare".'

In [15]:
import pandas as pd

df = pd.read_csv("teams2.csv")

In [16]:
df

,Contact Name,Project Title,Team Name,Mentor
0,Jamie Conlon,English - French Machine Translation,Merge Conflict,Lihua
1,Humza Saeed,Sentiment Analysis of Steam reviews,"""As an AI model",I am not able to generate a cool group name ...
2,Gerald Huang,Transforming Research Abstracts into Click-Wo...,Pre-trained Monkeys,Saurav
3,Tingyu Shi,Summarization and QA,chatOMG,Lihua
4,Li Mo,LLM Prompt Recovery,Language Alchemists,Xiangyu
5,Tan Hugh,Natural Language Logical Reasoning,ClosedAI,Saurav
6,Quoc Pham,Chatbot like tutor tailored for University co...,Sentient Sentences,Lihua
7,Rifa Jamal,UniPal,Natural Language Professionals,Xiangyu
8,Evan Li,Clinical Trial Report Statements Inference,Autoencoders Roll Out,Saurav
9,Vincent Morcilla,Medical Chatbot based on Llama2-Chat-13b,MedBot,Lihua


In [19]:
from langchain.vectorstores import Chroma
embedding_function = OpenAIEmbeddings(openai_api_key = key)
db = Chroma.from_documents(data, embedding_function)

In [20]:
retriever = db.as_retriever()

In [21]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(openai_api_key = key)

In [22]:
chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
    | StrOutputParser()
)

In [23]:
chain.invoke({"question": "Which project is 'Sentient Sentences' working on?"})

"The project 'Sentient Sentences' is working on is a Chatbot like tutor tailored for University courses."

In [24]:
chain.invoke({"question": "Which project is Tingyu Shi working on?"})

'Tingyu Shi is working on the project titled "Summarization and QA" with the team named "chatOMG".'

In [25]:
chain.invoke({"question": "How many teams in total is Lihua mentoring?"}) # Not Correct

'Lihua is mentoring 2 teams in total.'

In [27]:
chain.invoke({"question": "What are the project titles involving medical or similar data?"})

'1. Project Title: Medical Chatbot based on Llama2-Chat-13b\n2. Project Title: Clinical Trial Report Statements Inference'

In [103]:
chain.invoke({"question": "What are the names of the mentors?"})

'Saurav and Lihua'

In [104]:
chain.invoke({"question": "What are the names of the three mentors?"})

'The names of the three mentors are Saurav, Lihua, and Lihua.'

In [105]:
# See more loaders here: https://python.langchain.com/docs/modules/data_connection/document_loaders/